Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Module Project: Regression II

In this project, you'll continue working with the New York City rent dataset you used in the last module project.

## Directions

The tasks for this project are as follows:

- **Task 1:** Import `csv` file using `wrangle` function.
- **Task 2:** Conduct exploratory data analysis (EDA), and modify `wrangle` function to engineer two new features.
- **Task 3:** Split data into feature matrix `X` and target vector `y`.
- **Task 4:** Split feature matrix `X` and target vector `y` into training and test sets.
- **Task 5:** Establish the baseline mean absolute error for your dataset.
- **Task 6:** Build and train a `Linearregression` model.
- **Task 7:** Calculate the training and test mean absolute error for your model.
- **Task 8:** Calculate the training and test $R^2$ score for your model.
- **Stretch Goal:** Determine the three most important features for your linear regression model.

**Note**

You should limit yourself to the following libraries for this project:

- `matplotlib`
- `numpy`
- `pandas`
- `sklearn`

In [ ]:
import pandas as pd
import numpy as np
!pip install category_encoders==2.*
from category_encoders import OneHotEncoder

# I. Wrangle Data

In [ ]:
def wrangle(filepath):
    df = pd.read_csv(filepath,
                     parse_dates=['created'],
                     index_col='created')
    df.dropna(inplace=True)
    
    # Remove the most extreme 1% prices,
    # the most extreme .1% latitudes, &
    # the most extreme .1% longitudes
    df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
            (df['price'] <= np.percentile(df['price'], 99.5)) & 
            (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
            (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
            (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
            (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
    
    #Adding New features
    df['total_perks'] = df[df.columns[9:]].sum(axis=1)
    df['total_rooms'] = df['bathrooms'] + df['bedrooms']

    df.drop(columns=['description',	'display_address',	'street_address'], inplace=True)

    # Instantiating the encoder as an object and
    # create columns Using the fit_transform method
    encoder = OneHotEncoder(use_cat_names=True)
    encoder.fit(df)
    df = encoder.transform(df) 

    return df

filepath = DATA_PATH + 'apartments/renthop-nyc.csv'

**Task 1:** Add the following functionality to the above `wrangle` function.

- The `'created'` column will parsed as a `DateTime` object and set as the `index` of the DataFrame. 
- Rows with `NaN` values will be dropped.

Then use your modified function to import the `renthop-nyc.csv` file into a DataFrame named `df`.

In [ ]:
df = wrangle(filepath)

**Task 2:** Using your `pandas` and dataviz skills decide on two features that you want to engineer for your dataset. Next, modify your `wrangle` function to add those features. 

**Note:** You can learn more about feature engineering [here](https://en.wikipedia.org/wiki/Feature_engineering). Here are some ideas for new features:

- Does the apartment have a description?
- Length of description.
- Total number of perks that apartment has.
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths).

In [ ]:
# Conduct your exploratory data analysis here, 
# and then modify the function above.

df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 47260 entries, 2016-06-24 07:54:24 to 2016-04-12 02:48:07
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   bathrooms              47260 non-null  float64
 1   bedrooms               47260 non-null  int64  
 2   latitude               47260 non-null  float64
 3   longitude              47260 non-null  float64
 4   price                  47260 non-null  int64  
 5   interest_level_medium  47260 non-null  int64  
 6   interest_level_low     47260 non-null  int64  
 7   interest_level_high    47260 non-null  int64  
 8   elevator               47260 non-null  int64  
 9   cats_allowed           47260 non-null  int64  
 10  hardwood_floors        47260 non-null  int64  
 11  dogs_allowed           47260 non-null  int64  
 12  doorman                47260 non-null  int64  
 13  dishwasher             47260 non-null  int64  
 14  no_fee             

,bathrooms,bedrooms,latitude,longitude,price,interest_level_medium,interest_level_low,interest_level_high,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_perks,total_rooms
count,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.00000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000,47260.000000
mean,1.203364,1.544012,40.750689,-73.972736,3578.041536,0.234596,0.688574,0.076830,0.521075,0.474884,0.492044,0.443377,0.426153,0.424799,0.377359,0.052751,0.268240,0.191727,0.180470,0.137050,0.141832,0.106073,0.090055,0.060559,0.057025,0.053618,0.04744,0.032628,0.044033,0.040626,0.028121,0.026280,4.718218,2.747376
std,0.472804,1.108318,0.038839,0.028964,1765.210584,0.423750,0.463081,0.266325,0.499561,0.499374,0.499942,0.496789,0.494522,0.494318,0.484731,0.223538,0.443048,0.393663,0.384582,0.343904,0.348882,0.307934,0.286264,0.238521,0.231893,0.225265,0.21258,0.177663,0.205171,0.197425,0.165320,0.159969,3.439983,1.414901
min,0.000000,0.000000,40.575700,-74.087300,1375.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,40.728200,-73.991800,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
50%,1.000000,1.000000,40.751600,-73.978000,3150.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2.000000
75%,1.000000,2.000000,40.773900,-73.955000,4095.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,4.000000
max,10.000000,8.000000,40.989400,-73.700100,15500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,19.000000,12.000000


In [ ]:
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level_medium,interest_level_low,interest_level_high,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_perks,total_rooms
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,3000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,5465,0,1,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3.0
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,2850,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2.0
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,3275,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2.0
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,3350,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5.0


# II. Split Data

**Task 3:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'price'`.

**Note:** In contrast to the last module project, this time you should include _all_ the numerical features in your dataset.

In [ ]:
X = df.drop(columns=['price'])
y = df['price']

**Task 4:** Split `X` and `y` into a training set (`X_train`, `y_train`) and a test set (`X_test`, `y_test`).

- Your training set should include data from April and May 2016. 
- Your test set should include data from June 2016.

In [ ]:
mask = (df.index >= "2016-04-01") & (df.index < "2016-06-01")

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

# III. Establish Baseline

**Task 5:** Since this is a **regression** problem, you need to calculate the baseline mean absolute error for your model. First, calculate the mean of `y_train`. Next, create a list `y_pred` that has the same length as `y_train` and where every item in the list is the mean. Finally, use `mean_absolute_error` to calculate your baseline.

In [ ]:
from sklearn.metrics import mean_absolute_error

y_pred = [y_train.mean()] * len(y_train)

baseline_mae = mean_absolute_error(y_train, y_pred)
print('Baseline MAE:', baseline_mae)

Baseline MAE: 1202.398300781848


# IV. Build Model

**Task 6:** Build and train a `LinearRegression` model named `model` using your feature matrix `X_train` and your target vector `y_train`.

In [ ]:
# Step 1: Import predictor class
from sklearn.linear_model import LinearRegression

# Step 2: Instantiate predictor
model = LinearRegression()

# Step 3: Fit predictor on the (training) data
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# V. Check Metrics

**Task 7:** Calculate the training and test mean absolute error for your model.

In [ ]:
training_mae = mean_absolute_error(y_train, model.predict(X_train))
test_mae = mean_absolute_error(y_test, model.predict(X_test))

print('Training MAE:', training_mae)
print('Test MAE:', test_mae)

Training MAE: 672.9806423897443
Test MAE: 675.5598675712224


**Task 8:** Calculate the training and test $R^2$ score for your model.

In [ ]:
training_r2 = model.score(X_train, y_train)
test_r2 = model.score(X_test, y_test)

print('Training MAE:', training_r2)
print('Test MAE:', test_r2)

Training MAE: 0.6369150317981817
Test MAE: 0.6490664473526142


# VI. Communicate Results

**Stretch Goal:** What are the three most influential coefficients in your linear model? You should consider the _absolute value_ of each coefficient, so that it doesn't matter if it's positive or negative.

In [ ]:
absolutes = np.abs(model.coef_)
best_cofficients = sorted(absolutes)[-3:]

In [ ]:
# Import the feature selector utility
from sklearn.feature_selection import SelectKBest, f_regression

# Create the selector object with the best k=1 features
selector = SelectKBest(score_func=f_regression, k=3)

# Run the selector on the training data
X_train_selected = selector.fit_transform(X_train, y_train)

# Find the features that was selected
selected_mask = selector.get_support()
all_features = X_train.columns
selected_feature = all_features[selected_mask]

print('The 3 Best Features are: ', list(selected_feature))
print('Best Features Coefficients:', best_cofficients)

The 3 Best Features are:  ['bathrooms', 'bedrooms', 'total_rooms']
Best Features Coefficients: [900195944122016.4, 900195944123227.8, 900195944123719.9]
